In [94]:
import bz2
import subprocess
import xml.sax
import mwparserfromhell
import networkx as nx
import matplotlib.pyplot as plt
import gc
import json
import os
from functools import partial
from multiprocessing import Pool 
import tqdm 
from timeit import default_timer as timer

In [95]:
class WikiXmlHandler(xml.sax.handler.ContentHandler):
    """Content handler for Wiki XML data using SAX"""
    def __init__(self):
        xml.sax.handler.ContentHandler.__init__(self)
        self._buffer = None
        self._values = {}
        self._current_tag = None
        self._pages = []

    def characters(self, content):
        """Characters between opening and closing tags"""
        if self._current_tag:
            self._buffer.append(content)

    def startElement(self, name, attrs):
        """Opening tag of element"""
        if name in ('title', 'text'):
            self._current_tag = name
            self._buffer = []

    def endElement(self, name):
        """Closing tag of element"""
        if name == self._current_tag:
            self._values[name] = ' '.join(self._buffer)

        if name == 'page':
            self._pages.append((self._values['title'], self._values['text']))

In [96]:
def find_links(data_path, save = True):
    handler = WikiXmlHandler()
    parser = xml.sax.make_parser()
    parser.setContentHandler(handler)

    for i, line in enumerate(subprocess.Popen(['bzcat'], 
                             stdin = open(f'./partitioned-data/{data_path}'), 
                             stdout = subprocess.PIPE).stdout):
        try:
            parser.feed(line)
        except StopIteration:
            break
    
    if save:
        new_data_path = data_path[0:data_path.find('xml')-1] + '-' + data_path[data_path.find('xml') + 4:-4]
        m = {}
        for page in handler._pages:
            wikipage = mwparserfromhell.parse(page[1])
            wikilinks = [str(x.title) for x in wikipage.filter_wikilinks()]
            m[page[0]] = wikilinks
            
        with open(f'./json-data/{new_data_path}.json', 'w') as fout:
            fout.write(json.dumps(m))

        partition_dir = './json-data'
        print(f'{ len(os.listdir(partition_dir)) } files processed.', end = '\r')

    # Memory management
    del handler
    del parser
    gc.collect()
    return None

In [102]:
partitions = os.listdir('./partitioned-data')
partitions.remove('.DS_Store')
for data_path in partitions:
    if data_path[0:data_path.find('xml')-1] + '-' + data_path[data_path.find('xml') + 4:-4] + '.json' in os.listdir('./json-data'):
        partitions.remove(data_path)
        
partitions = sorted(partitions, key = lambda x: os.path.getsize(f'./partitioned-data/{x}'))
partitions = partitions[0:5]
partitions

['enwiki-latest-pages-articles26.xml-p42567203p42663461.bz2',
 'enwiki-latest-pages-articles24.xml-p33503451p33952815.bz2',
 'enwiki-latest-pages-articles23.xml-p29823661p30503450.bz2',
 'enwiki-latest-pages-articles1.xml-p1p30303.bz2',
 'enwiki-latest-pages-articles20.xml-p20254736p21222156.bz2']

In [ ]:
# Create a pool of workers to execute processes
pool = Pool(processes = 8)

start = timer()

# Map (service, tasks), applies function to each partition
results = pool.map(find_links, partitions)

pool.close()
pool.join()

end = timer()
print(f'{end - start} seconds elapsed.')